In [1]:
import pandas as pd
from transformers import Wav2Vec2Processor
from datasets import load_metric, Dataset
import numpy as np
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer

import os

os.environ["CUDA_VISIBLE_DEVICES"]=""


In [4]:
train_data1 = pd.read_pickle('./train_data1.pkl')[['input_values','labels']]

In [5]:
train_data2 = pd.read_pickle('./train_data2.pkl')[['input_values','labels']]

In [6]:
train_data = Dataset.from_pandas( pd.concat([train_data1,train_data2]))

In [7]:
del train_data1
del train_data2

In [8]:
train_data

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 27578
})

In [9]:
import gc
# del train_data1
# del train_data2
gc.collect()

10

In [10]:
test_data = Dataset.from_pandas(pd.read_pickle('./test_data_final.pkl')[['input_values','labels']])

In [11]:
processor = Wav2Vec2Processor.from_pretrained('./icfoss-model')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[Code for collator](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81)

In [12]:
import torch
import jiwer
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [14]:
wer_metric = load_metric("wer")
# wer_metric

C:\Users\unmuk\AppData\Local\Temp\ipykernel_15280\1284616902.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
Using the latest cached version of the module from C:\Users\unmuk\.cache\huggingface\modules\datasets_modules\metrics\wer\959256dcd2f8f341e108aa768b4433db0dd58f85e23e818e491c2d6d0f84113b (last modified on Sat Feb 25 17:29:22 2023) since it couldn't be found locally at wer, or remotely on the Hugging Face Hub.


In [15]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.094,
    hidden_dropout=0.05,
    feat_proj_dropout=0.0,
    mask_time_prob=0.082,
    layerdrop=0.045,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)


In [ ]:
model.freeze_feature_encoder()


In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='/working/',
  group_by_length=False,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=4,
  gradient_checkpointing=True,
  evaluation_strategy="steps",
  num_train_epochs=1,
  save_steps=800,
  eval_steps=500,
  logging_steps=500,
  learning_rate=2.4e-4,
  warmup_steps=800,
  save_total_limit=3,
    fp16=True,
    optim="adafactor",
    overwrite_output_dir=False
)

import os
os.environ['USE_FP16'] = 'true'

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=processor.feature_extractor,
)


# _Training_

In [ ]:
trainer.train

In [ ]:
trainer.save_model(f"./saved/")